In [1]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import pickle
from numpy.linalg import eigvals, eig

def f_deltar(deltar, J, delta, g, x):
    return delta * np.exp(-2 * np.sum(f_f(deltar, J, g, x) ** 2))

def f_f(deltar, J, g, x):
    return g / (np.sqrt(N) * w) * (f_E(deltar, J) +  J * np.cos(2 * np.pi * ks * x / N)) / (f_E(deltar, J) +  J * np.cos(2 * np.pi * ks * x / N) + (deltar ** 2) / w)

def f_J(deltar, J, delta, g, x):
    return 2 * np.sum(f_f(deltar, J, g, x) * (2 * g / np.sqrt(N) - w * f_f(deltar, J, g, x)) * np.cos(2 * np.pi * ks * x / N))

def f_E(deltar, J):
    return np.sqrt(deltar ** 2 + J ** 2)

def f_analisis(deltas, gs, xs):
    analisis = {}
    
    for x in xs:
        x = round(x, 2)
        analisis[x] = {}
        
        deltar = 0.001
        J = 0.001

        for delta in deltas:
            delta = round(delta, 2)
            analisis[x][delta] = {}

            for g in gs:
                g = round(g, 4)

                while True:
                    change = max(abs(deltar - f_deltar(deltar, J, delta, g, x)), abs(J - f_J(deltar, J, delta, g, x)))

                    deltar = f_deltar(deltar, J, delta, g, x)
                    J = f_J(deltar, J, delta, g, x)

                    if change < 1e-9: #cuando la mejora es ya muy pequeña cierro el bucle
                        break

                analisis[x][delta][g] = (deltar, J)

    return analisis

def f_cos(deltar, J):
    num = deltar + f_E(deltar, J)
    den = np.sqrt(num ** 2 + J ** 2)
    
    return num / den

def f_sin(deltar, J):
    num  = J
    den  = np.sqrt((deltar + f_E(deltar, J)) ** 2 + J ** 2)
    
    return num / den

def eff_hamiltonian(deltar, J, g, x):
    cos = f_cos(deltar, J)
    sin = f_sin(deltar, J)
    factor = cos ** 2 - sin ** 2
    
    h1 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    for k in range(N):
        h1[2 + k, 2 + k] = -deltar * factor
        
    h2 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    for k in range(N):
        h2[2 + k, 2 + k] = w[k]
        
    h3 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    f0 = f_f(deltar, 0, g, x) 
    f = f_f(deltar, J, g, x) 
    deltaf = f - f0
    for k in range(N):
        h3[k + 2, 0] = (2 * deltar * f0[k] + deltaf[k] * (deltar - w[k])) * cos * np.exp(1j * 2 * np.pi * ks[k] * x / N)
        h3[0, k + 2] = np.conjugate(h3[k + 2, 0])
        h3[k + 2, 1] = (2 * deltar * f0[k] + deltaf[k] * (deltar - w[k])) * cos
        h3[1, k + 2] = np.conjugate(h3[k + 2, 1])
        
    h4 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    for k in range(N):
        for p in range(N):
            h4[k + 2, p + 2] = 2 * deltar * f[k] * f[p] * factor * (1 + np.exp(1j * 2 * np.pi * (ks[k] - ks[p]) * x / N))
            
    h5 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    h5[1, 0] = -J
    h5[0, 1] = -J
    for k in range(N):
        h5[2 + k, 2 + k] = -J * 2 * cos * sin
        
    return h1 + h2 + h3 + h4 + h5

def rwa_hamiltonian(delta, g, x):
    c = g / np.sqrt(N)
    
    h1 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    for k in range(N):
        h1[k + 2, k + 2] = -delta
        
    h2 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    for k in range(N):
        h2[k + 2, k + 2] = w[k]
    
    h3 = np.zeros(shape = (N + 2, N + 2), dtype = np.complex_)
    for k in range(N):
        h3[k + 2, 0] = c * np.exp(1j * 2 * np.pi * ks[k] * x / N)
        h3[0, k + 2] = np.conjugate(h3[k + 2, 0])
        h3[k + 2, 1] = c
        h3[1, k + 2] = c
        
    return h1 + h2 + h3

def sorted_eigsystem(H):
    vals, vects = eig(H)
    idx = np.argsort(vals)
    vals = vals[idx]
    vects = vects[:,idx]
    
    return Eigensystem(vals, vects)

class Eigensystem:
    def __init__(self, vals, vects):
        self.vals = vals
        self.vects = vects
        self.size = len(vals)

def f_eigenanalisis(analisis):
    eigenanalisis_eff = {}
    eigenanalisis_rwa = {}
    
    for x in analisis.keys():
        eigenanalisis_eff[x] = {}
        eigenanalisis_rwa[x] = {}
        
        for delta in analisis[x].keys():
            eigenanalisis_eff[x][delta] = {}
            eigenanalisis_rwa[x][delta] = {}

            for g in analisis[x][delta].keys():
                H_eff = eff_hamiltonian(analisis[x][delta][g][0], analisis[x][delta][g][1], g, x)
                H_rwa = rwa_hamiltonian(delta, g, x)

                eig_eff = sorted_eigsystem(H_eff)
                eig_rwa = sorted_eigsystem(H_rwa)

                eigenanalisis_eff[x][delta][g] = eig_eff
                eigenanalisis_rwa[x][delta][g] = eig_rwa
            
    return eigenanalisis_eff, eigenanalisis_rwa

In [2]:
w0 = 1.0
gamma = 0.4
N = 100
ks = np.arange(N)
w = w0 - 2 * gamma * np.cos(2 * np.pi / N * ks)
xs = np.arange(0.0, 21.0, 1.0)
deltas = np.arange(0.1, 2.1, 0.1)
gs = np.arange(0.0, 0.51, 0.01)

In [3]:
analisis = f_analisis(deltas, gs, xs)

f = open('analisis/analisis.pk', 'wb')
pickle.dump(analisis, f)
f.close()

In [4]:
gs = np.arange(0.0, 0.32, 0.02)

analisis = f_analisis(deltas, gs, xs)

In [5]:
eigen_eff, eigen_rwa = f_eigenanalisis(analisis)

f = open('analisis/eigenanalisis_eff.pk', 'wb')
pickle.dump(eigen_eff, f)
f.close()

f = open('analisis/eigenanalisis_rwa.pk', 'wb')
pickle.dump(eigen_rwa, f)
f.close()

In [ ]:
gs = np.arange(0.0, 0.32, 0.02)

analisis = f_analisis(deltas, gs, xs)
eigen_eff, eigen_rwa = f_eigenanalisis(analisis)